In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import os
import json
from datetime import datetime

# Define the file path for storing expenses
EXPENSE_FILE = "expenses.json"

# Load existing expenses from file
def load_expenses():
    if os.path.exists(EXPENSE_FILE):
        with open(EXPENSE_FILE, 'r') as file:
            return json.load(file)
    return []

# Save expenses to file
def save_expenses(expenses):
    with open(EXPENSE_FILE, 'w') as file:
        json.dump(expenses, file)

# Add a new expense
def add_expense():
    amount = amount_var.get()
    category = category_var.get()
    description = description_var.get()
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if not amount or not category or not description:
        messagebox.showwarning("Input Error", "All fields are required.")
        return
    
    try:
        amount = float(amount)
    except ValueError:
        messagebox.showwarning("Input Error", "Please enter a valid amount.")
        return
    
    expense = {
        "date": date,
        "amount": amount,
        "category": category,
        "description": description
    }
    
    expenses.append(expense)
    save_expenses(expenses)
    messagebox.showinfo("Success", "Expense added successfully!")
    amount_var.set("")
    category_var.set("")
    description_var.set("")

# List all expenses
def list_expenses():
    list_window = tk.Toplevel(root)
    list_window.title("List of Expenses")
    
    tree = ttk.Treeview(list_window, columns=("Date", "Amount", "Category", "Description"), show='headings')
    tree.heading("Date", text="Date")
    tree.heading("Amount", text="Amount")
    tree.heading("Category", text="Category")
    tree.heading("Description", text="Description")
    
    for expense in expenses:
        tree.insert("", "end", values=(expense["date"], expense["amount"], expense["category"], expense["description"]))
    
    tree.pack(expand=True, fill='both')

# Calculate total expenses for a specified time frame
def calculate_total_expenses():
    timeframe = timeframe_var.get()
    total = 0
    now = datetime.now()
    
    for expense in expenses:
        expense_date = datetime.strptime(expense['date'], "%Y-%m-%d %H:%M:%S")
        if timeframe == "Daily" and expense_date.date() == now.date():
            total += expense['amount']
        elif timeframe == "Weekly" and (now - expense_date).days < 7:
            total += expense['amount']
        elif timeframe == "Monthly" and expense_date.month == now.month and expense_date.year == now.year:
            total += expense['amount']
    
    messagebox.showinfo("Total Expenses", f"Total {timeframe.lower()} expenses: {total}")

# Generate a monthly report
def generate_monthly_report():
    report = {}
    now = datetime.now()
    
    for expense in expenses:
        expense_date = datetime.strptime(expense['date'], "%Y-%m-%d %H:%M:%S")
        if expense_date.month == now.month and expense_date.year == now.year:
            category = expense['category']
            if category not in report:
                report[category] = 0
            report[category] += expense['amount']
    
    report_window = tk.Toplevel(root)
    report_window.title("Monthly Report")
    
    if not report:
        tk.Label(report_window, text="No expenses recorded for this month.").pack()
        return
    
    for category, total in report.items():
        tk.Label(report_window, text=f"Category: {category}, Total: {total}").pack()

# Main GUI setup
root = tk.Tk()
root.title("Expense Tracker")

expenses = load_expenses()

amount_var = tk.StringVar()
category_var = tk.StringVar()
description_var = tk.StringVar()
timeframe_var = tk.StringVar(value="Daily")

tk.Label(root, text="Amount:").grid(row=0, column=0)
tk.Entry(root, textvariable=amount_var).grid(row=0, column=1)

tk.Label(root, text="Category:").grid(row=1, column=0)
tk.Entry(root, textvariable=category_var).grid(row=1, column=1)

tk.Label(root, text="Description:").grid(row=2, column=0)
tk.Entry(root, textvariable=description_var).grid(row=2, column=1)

tk.Button(root, text="Add Expense", command=add_expense).grid(row=3, column=0, columnspan=2)

tk.Button(root, text="List Expenses", command=list_expenses).grid(row=4, column=0, columnspan=2)

tk.Label(root, text="Total Expenses:").grid(row=5, column=0)
ttk.Combobox(root, textvariable=timeframe_var, values=["Daily", "Weekly", "Monthly"]).grid(row=5, column=1)
tk.Button(root, text="Calculate", command=calculate_total_expenses).grid(row=6, column=0, columnspan=2)

tk.Button(root, text="Generate Monthly Report", command=generate_monthly_report).grid(row=7, column=0, columnspan=2)

tk.Button(root, text="Exit", command=root.quit).grid(row=8, column=0, columnspan=2)

root.mainloop()


